<a href="https://colab.research.google.com/github/JavaGenbu/Machine-Learning-JoseValero/blob/main/M02-Aprendizaje_supervisado/M2U2-Optimizaci%C3%B3n_por_descenso_de_gradiente/M2U2-7-Tasaci%C3%B3n_de_viviendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión lineal: Ejemplo de tasación de viviendas
M2U2 - Ejercicio 7

## ¿Qué vamos a hacer?
- Entrenar un modelo de regresión lineal multivariable
- Crear un dataset sintético siguiendo un esquema de datos real

Recuerda seguir las instrucciones para las entregas de prácticas indicadas en [Instrucciones entregas](https://github.com/Tokio-School/Machine-Learning/blob/main/Instrucciones%20entregas.md).

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt

## Set de datos de tasación de viviendas sintético

En esta ocasión vamos a explorar cómo crear un dataset sintético que siga la estructura que queramos, para simular un dataset real con plena flexibilidad.

En este caso, vamos a usar como ejemplo un dataset inmobiliario con el objetivo de entrenar un tadador de viviendas con las siguientes características:

Variable objetivo o dependiente:
- Precio de venta (int)

Características, variables explicatorias o independientes:
- Superficie útil (int)
- Localización (int, representando la barriada como categoría ordinal)
- Tipo de vivienda (int, representando piso, chalet, adosado, ático, etc. como categoría ordinal)
- Nº de habitaciones (int)
- Nº de cuartos de baño (int)
- Garaje (int, 0/1 representando si tiene o no)
- Superficie zonas comunes (int)
- Año de construcción (int)

Nuestro modelo intentará aproximar una función lineal multivariable que nos permita interpretar el mercado inmobiliario y hacer predicciones sobre el precio de venta de nuevas viviendas, según la función lineal:

$$Y = h_\Theta(x) = X \times \Theta^T$$
Donde $h_\Theta(x)$ es la hipótesis lineal.

### Creación del dataset sintético

Primero crearemos un ejemplo de una vivienda con datos conocidos, con los valores de sus características y el precio de venta:

In [ ]:
x_ej1 = np.asarray([100, 4, 2, 2, 1, 0, 30, 10])
y_ej1 = np.asarray([80000])

print('Precio de venta de la vivienda:', y_ej1[0])
print('Superficie útil:', x_ej1[0])
print('Localización:', x_ej1[1])
print('Tipo de vivienda:', x_ej1[2])
print('Nº de habitaciones:', x_ej1[3])
print('Nº de cuartos de baño:', x_ej1[4])
print('Garaje (sí/no):', x_ej1[5])
print('Superficie de zonas comunes:', x_ej1[6])
print('Antigüedad:', x_ej1[7])

De esta forma podemos crear nuevos ejemplos con los valores que queramos a nuestro antojo.

Modifica los valores de la vivienda anterior para generar otras viviendas de forma manual.

Del mismo modo que hemos creado un ejemplo de vivienda manualmente, vamos a crear múltiples ejemplos aleatorios de forma automática:

*Nota*: Por simpleza a la hora de generar nºs aleatorios con código, vamos a usar float en lugar de int en los mismos rangos lógicos para las características de las viviendas.

In [ ]:
m = 100           # nº de ejemplos de viviendas
n = x_ej1.shape[0]    # nº de características de cada vivienda

X = np.random.rand(m, n)

print('Primeros 10 ejemplos de X:')
print(X[:10, :])
print('Tamaño de la matriz de ejemplos X:')
print(X.shape)

*¿Cómo podemos crear el vector *Y* de precios de venta de nuestro dataset sintético, de forma que sigua la relación lineal que queremos aproximar?*

Para ello, debemos partir de una *Theta_verd* como en ejercicios pasados.

A veces, el problema es obtener una *Y* en el rango que quisiéramos modificando cada valor de *Theta_verd*, lo que puede ser bastante tedioso.

En otros casos, una alternativa sería crear *X* e *Y* manualmente, y luego calcular la *Theta_verd* correspondinete a dichas matrices.

En este caso, crearemos *Theta_verd* a mano para poder interpretar sus valores:

In [ ]:
x = X[0, :]

print('Ej. de vivienda con características aleatorias:')
print(x)

Theta_verd = np.asarray([1000., -500, 10000, 5000, 2500, 6000, 50, -1500])

print('\nEj. de pesos de características creados manualmente:')
print(Theta_verd)
print(Theta_verd.shape)

print('\nEl precio de venta de dicha vivienda viene dado por sus características:')
print('Por cada m2 de superficie útil:', Theta_verd[0])
print('Por cada km de distancia al centro:', Theta_verd[1])
print('Según el tipo de vivienda:', Theta_verd[2])
print('Según el nº de habitaciones:', Theta_verd[3])
print('Según el nº de cuartos de baño:', Theta_verd[4])
print('Según si tiene garaje:', Theta_verd[5])
print('Por cada m2 de superficie de zonas comunes:', Theta_verd[6])
print('Por cada año de antigüedad:', Theta_verd[7])

Cada uno de estos pesos multiplicará su característica correspondiente, sumando o restando al precio total de la vivienda.

Sin embargo, a nuestro set de datos sintético ideal le falta un término muy importante, el _bias_ o _intercept_:
El _bias_ es el término _b_ de cualquier recta $y = m * x \times b$, el que respresenta la suma de todas las constantes que afectan a nuestro modelo o el precio base, antes de verse modificado por el resto de características.

Este _bias_ es muy importante porque un modelo no sólo debe poder aproximar pesos o coeficientes que multipliquen a las características dadas, sino también a valores constantes que no dependan de las características concretas de cada ejemplo.

O lo que es lo mismo: _precio = coeficientes * características + "bias o precio base"_.

P. ej., en el caso de viviendas, sería el precio de partida de todas las viviendas, si lo hubiera, independientemente de sus características, las cuales suman o restan a partir del mismo. En el caso de un estudio sin habitaciones independientes, WC compartido, sin garaje, etc., etc., donde todas sus características fueran 0, nos permitiría determinar su precio de venta, *que no sería de 0 € seguramente*.

Añadimos a *Theta_verd* un _bias_ o precio de partida.

In [ ]:
# CUIDADO: Cada vez que ejecutamos esta celda estamos añadiendo una columna
# de 1s a Theta_verd y X, por lo que sólo debemos ejecutarla una vez

Theta_verd = np.insert(Theta_verd, 0, 25000)    # 25000 € de precio de partida = theta[0]
X = np.insert(X, 0, np.ones(m), axis=1)

print('Theta verdadero y 10 primeros ejemplos (filas) de X:')
print(Theta_verd)
print(X[:10, :])
print('Tamaños de X y Tetha verdadero:')
print(X.shape)
print(Theta_verd.shape)

Contando con dicho *Theta_verd*, podemos establecer el vector *Y* de precios de venta de nuestros ejemplos:

In [ ]:
# TODO: Modifica el siguiente código para añadir un término de error aleatorio a Y

error = 0.15

Y = np.matmul(X, Theta_verd)

print('Precios de venta:')
print(Y)
print(Y.shape)

*Nota*: Al no haber usado valores int finalmente, los precios de venta son también valores float.

## Entrenamiento del modelo

Una vez listo nuestro dataset de datos de entrenamiento, vamos a entrenar el modelo de regresión lineal.

Para ello, copia las celdas correspondientes de los últimos ejercicios para entrenar el modelo con estos datos y evalúa su comportamiento:

In [ ]:
# TODO: Copia las celdas correspondientes para entrenar un modelo por regresión lineal y evaluar su entrenamiento

# Dividimos los datos en conjunto de entrenamiento y de test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Entrenamos el modelo de regresión lineal
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X_train, y_train)

# Evaluamos el modelo en el conjunto de entrenamiento
train_score = reg.score(X_train, y_train)
print('R2 score en conjunto de entrenamiento:', train_score)

# Evaluamos el modelo en el conjunto de test
test_score = reg.score(X_test, y_test)
print('R2 score en conjunto de test:', test_score)

## Predicciones 

Por tanto, si creamos un nuevo ejemplo de vivienda manualmente con características aleatorias, podemos hacer una predicción sobre su precio de venta:

In [ ]:
# Creamos una nueva vivienda con características aleatorias
x_pred = np.random.rand(n+1)
x_pred[0] = 1    # Añadimos el término de bias de 1

# Calculamos su precio predicho usando el modelo entrenado
y_pred = np.dot(x_pred, reg.coef_) + reg.intercept_

print('Ej. de vivienda aleatoria:')
print(x_pred)

print('Precio predicho para dicha vivienda:')
print(y_pred)

¿Y nuestro dataset sintético original? ¿Qué precio de venta tendría? ¿Y qué resíduos tendría nuestro modelo sobre ellos?

In [ ]:
# Calculamos las predicciones del modelo para todos los ejemplos de entrenamiento
Y_pred = np.matmul(X, theta)

# Calculamos los residuos
residuos = Y - Y_pred

# Representamos gráficamente los residuos
plt.figure()
plt.title('Residuos del modelo')
plt.xlabel('Ejemplo de entrenamiento')
plt.ylabel('Residuo')
plt.scatter(range(len(residuos)), residuos)
plt.show()